# 使用KNN识别异常网络连接

我们使用 NSL-KDD 入侵检测数据集为例，说明使用KNN算法识别异常网络连接的基本思路。

## 数据准备


In [23]:
import pandas as pd

df = pd.read_csv("../lectures/data/NSL-KDD/KDDTrain+_20Percent.txt",
                 names=['duration', 'protocol_type', 'service', 'flag', 'src_bytes',
                        'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot',
                        'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell',
                        'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
                        'num_access_files', 'num_outbound_cmds', 'is_host_login',
                        'is_guest_login', 'count', 'srv_count', 'serror_rate',
                        'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
                        'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
                        'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate',
                        'dst_host_same_src_port_rate',
                        'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
                        'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
                        'dst_host_srv_rerror_rate', 'attack_type', 'difficulty_level'])

df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack_type,difficulty_level
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21


In [24]:
df.tail()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack_type,difficulty_level
25187,0,tcp,exec,RSTO,0,0,0,0,0,0,...,0.03,0.06,0.00,0.00,0.0,0.0,1.0,1.0,neptune,19
25188,0,tcp,ftp_data,SF,334,0,0,0,0,0,...,1.00,0.00,1.00,0.18,0.0,0.0,0.0,0.0,warezclient,12
25189,0,tcp,private,REJ,0,0,0,0,0,0,...,0.05,0.07,0.00,0.00,0.0,0.0,1.0,1.0,neptune,21
25190,0,tcp,nnsp,S0,0,0,0,0,0,0,...,0.08,0.06,0.00,0.00,1.0,1.0,0.0,0.0,neptune,20
25191,0,tcp,finger,S0,0,0,0,0,0,0,...,0.19,0.03,0.01,0.00,1.0,1.0,0.0,0.0,neptune,18


## 观察数据分布

In [25]:
df.describe()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,difficulty_level
count,25192.000000,2.519200e+04,2.519200e+04,25192.000000,25192.000000,25192.00000,25192.000000,25192.000000,25192.000000,25192.000000,...,25192.000000,25192.000000,25192.000000,25192.000000,25192.000000,25192.000000,25192.000000,25192.000000,25192.000000,25192.000000
mean,305.054104,2.433063e+04,3.491847e+03,0.000079,0.023738,0.00004,0.198039,0.001191,0.394768,0.227850,...,115.063036,0.519791,0.082539,0.147453,0.031844,0.285800,0.279846,0.117800,0.118769,19.487695
std,2686.555640,2.410805e+06,8.883072e+04,0.008910,0.260221,0.00630,2.154202,0.045418,0.488811,10.417352,...,110.646850,0.448944,0.187191,0.308367,0.110575,0.445316,0.446075,0.305869,0.317333,2.328541
min,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,10.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18.000000
50%,0.000000,4.400000e+01,0.000000e+00,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,61.000000,0.510000,0.030000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,20.000000
75%,0.000000,2.790000e+02,5.302500e+02,0.000000,0.000000,0.00000,0.000000,0.000000,1.000000,0.000000,...,255.000000,1.000000,0.070000,0.060000,0.020000,1.000000,1.000000,0.000000,0.000000,21.000000
max,42862.000000,3.817091e+08,5.151385e+06,1.000000,3.000000,1.00000,77.000000,4.000000,1.000000,884.000000,...,255.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,21.000000


## Rootkit相关属性选取与可视化分析

根据经验，Rootkit相关特征有：
- hot
- num_failed_logins
- logged_in
- num_compromised
- root_shell
- su_attempted
- num_root
- num_file_creations
- num_shells
- num_access_files
- num_outbound_cmds
- is_hot_login
- is_guest_login

我们可以将这部分特征提取出来

In [49]:

dfnew = df.loc[:,['hot','num_failed_logins','logged_in',
                     'num_compromised','root_shell','su_attempted','num_root',
                     'num_file_creations','num_shells','num_access_files',
                     'num_outbound_cmds','is_guest_login','attack_type']]
dfnew.head()

,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_guest_login,attack_type
0,0,0,0,0,0,0,0,0,0,0,0,0,normal
1,0,0,0,0,0,0,0,0,0,0,0,0,normal
2,0,0,0,0,0,0,0,0,0,0,0,0,neptune
3,0,0,1,0,0,0,0,0,0,0,0,0,normal
4,0,0,1,0,0,0,0,0,0,0,0,0,normal


## 数据预处理

### 将字符串类型数据变为数值型数据。

例如 attack_type

In [50]:
from sklearn.preprocessing import LabelEncoder

dfnew['attack_type'] = LabelEncoder().fit_transform(dfnew['attack_type'])
dfnew.head()

,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_guest_login,attack_type
0,0,0,0,0,0,0,0,0,0,0,0,0,11
1,0,0,0,0,0,0,0,0,0,0,0,0,11
2,0,0,0,0,0,0,0,0,0,0,0,0,9
3,0,0,1,0,0,0,0,0,0,0,0,0,11
4,0,0,1,0,0,0,0,0,0,0,0,0,11


### 将所有数值型数据进行归一化

为了平衡各量纲数据对结果的影响，需要将数据均平移至[0,1]区间中。



In [52]:
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler

#区间缩放，返回值为缩放到[0, 1]区间的数据
ndarray1 = MinMaxScaler().fit_transform(dfnew.iloc[:,:-1])
normed_XTrain = pd.DataFrame(ndarray1,columns=['hot','num_failed_logins','logged_in',
                     'num_compromised','root_shell','su_attempted','num_root',
                     'num_file_creations','num_shells','num_access_files',
                     'num_outbound_cmds','is_guest_login'])


normed_XTrain

C:\Users\leo\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_guest_login
0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0
1,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0
2,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0
3,0.000000,0.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0
4,0.000000,0.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0
5,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0
6,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0
7,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0
8,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0
9,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0


## 训练KNN模型


In [53]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors=5)

yTrain = dfnew.iloc[:,-1]
clf.fit(normed_XTrain,yTrain)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

## 交叉验证评分

In [54]:
from sklearn.model_selection import cross_val_score

cross_val_score(clf,normed_XTrain,yTrain,n_jobs=-1,cv=10)

C:\Users\leo\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


array([0.40023706, 0.72028526, 0.72391908, 0.70963903, 0.72698413,
       0.71099643, 0.71593166, 0.71451292, 0.71468365, 0.71804062])

## 重设超参n_neighbors

In [58]:
clf1 = KNeighborsClassifier(n_neighbors=10)
clf1.fit(normed_XTrain,yTrain)
cross_val_score(clf,normed_XTrain,yTrain,n_jobs=-1,cv=10)

C:\Users\leo\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


array([0.40023706, 0.72028526, 0.72391908, 0.70963903, 0.72698413,
       0.71099643, 0.71593166, 0.71451292, 0.71468365, 0.71804062])

## 预测

In [14]:
df_test = pd.read_csv("../lectures/data/NSL-KDD/KDDTest-21.txt",
                 names=['duration','protocol_type','service','flag','src_bytes','dst_bytes', \
                          'land','wrong_fragment','urgent','hot','num_failed_logins','logged_in', \
                          'num_compromised','root_shell','su_attempted','num_root','num_file_creations', \
                          'num_shells','num_access_files','num_outbound_cmds','is_hot_login', \
                          'is_guest_login','count','srv_count','serror_rate','srv_serror_rate', \
                          'rerror_rate','srv_rerror_rate','same_srv_rate','diff_srv_rate', \
                          'rv_diff_host_rate','dst_host_count','dst_host_srv_count', \
                          'dst_host_same_srv_rate','dst_host_diff_srv_rate','dst_host_same_src_port_rate', \
                          'dst_host_srv_diff_host_rate','dst_host_serror_rate','dst_host_srv_serror_rate', \
                          'dst_host_rerror_rate','dst_host_srv_rerror_rate'])

X_test = df_test.loc[:,['hot','num_failed_logins','logged_in','num_compromised',
                         'root_shell','su_attempted','num_root','num_file_creations',
                         'num_shells','num_access_files','num_outbound_cmds','is_hot_login',
                         'is_guest_login']]

ndarray2 = MinMaxScaler().fit_transform(df_test)
normed_dfXtest = pd.DataFrame(ndarray2,columns= [])
y_test=clf.predict(X_test)
y_test

array([16, 20, 21, ..., 18, 21, 20], dtype=int64)